In [83]:
#Importing the required 
%load_ext autoreload
%autoreload 1

import os 
os.environ['CUDA_VISIBLE_DEVICES']='1'


import numpy as np
import tensorflow as tf
import pandas as pd
import mdn
import IPython.display as ipd

from keras.optimizers import Adam

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model, model_from_json
from tensorflow.keras.layers import Embedding, Dense, Activation, LSTM, Dropout, Flatten
#from tensorflow.keras.optimizers import RMSprop, Adam
from keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing import image
from sklearn.metrics import mean_absolute_error, mean_squared_error
#from tensorflow.keras.models import Model, load_model, model_from_json
from keras.models import Model, load_model, model_from_json
from librosa.output import write_wav 
from sklearn.preprocessing import scale

%aimport SMCutils
su = SMCutils


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
#Load input data 
PATH = os.getcwd()
MotionPath = PATH+'/SMC_dataset1/motion_features9x5.csv'
MotionFeat = np.array(pd.read_csv(MotionPath, sep=','))

#Reshape into the correct format
MotionFeat = np.reshape(MotionFeat, (-1, 9, 5))

In [78]:
MotionFeat.shape
int(len(MotionFeat)/8)

150

In [79]:


#Load Model
N_MIXES = 10  # number of mixture components
OUTPUT_DIMS = 1024  # number of real-values predicted by each mixture component

model = load_model('MDN/1000Epochs.hdf5', custom_objects={'MDN': mdn.MDN, 'mdn_loss_func': mdn.get_mixture_loss_func(OUTPUT_DIMS,N_MIXES)})


In [80]:
MotionFeat.shape

(1201, 9, 5)

In [81]:
#Predict Distributions 
Results = np.zeros(shape=(120,8,1024))

for i in range(15):
    IndexStart = i*8
    IndexEnd = IndexStart+8
    Predicted = model.predict(MotionFeat[IndexStart:IndexEnd, :, :])
    
    #Sample Distributions
    y_sample = np.apply_along_axis(mdn.sample_from_output, 1, Predicted, 1024, N_MIXES, temp=1)
    y_sample = np.squeeze(y_sample)
    
    Results[i, :, :] = y_sample
    IndexStart += 8
    
np.save('Results/120Result', Results)

In [82]:
Results.shape

(120, 8, 1024)

In [84]:
#Send Results through decoder

ltsm_name = 'Results/120Result.npy'
x = np.load(ltsm_name)
element = np.random.randint(0,len(x))
x = np.reshape(x, (-1, 1024))
inputdata = x[1,:]
inputdata = np.reshape(inputdata, (1,-1))
print(x.shape, inputdata.shape)

(960, 1024) (1, 1024)


In [98]:
model_name    = 'lstm/20kconv.json'
weights_name  = 'lstm/20kconv.h5'
code_name     = 'lstm/ffff_20kconv_1024_03e_codedim1024_smallerconv.csv'

In [99]:
sr         = 22050
fmin       = 20
fmax       = sr / 2 
n_fft      = 4096
hop_length = 690
n_frames   = 128*2
n_mels     = 128

In [100]:
code_dim       = 1024
learning_rate  = 0.001
optimizer      = Adam(learning_rate=learning_rate)
loss           = 'mse'

In [101]:
json_file = open(model_name, 'r')
loaded_model_json = json_file.read()
json_file.close()
autoencoder = model_from_json(loaded_model_json)
autoencoder.load_weights(weights_name)
autoencoder.compile(loss=loss, optimizer=optimizer)

In [102]:
encoder = autoencoder.get_layer('encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_data (InputLayer)      (None, 32768)             0         
_________________________________________________________________
reshape_5 (Reshape)          (None, 128, 256, 1)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 128, 256, 32)      320       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 64, 128, 32)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 64, 128, 64)       18496     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 32, 64, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 32, 64, 128)       7385

In [103]:
decoder = autoencoder.get_layer('decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32768)             33587200  
Total params: 33,587,200
Trainable params: 33,587,200
Non-trainable params: 0
_________________________________________________________________


In [104]:
SpectrogramPath = PATH+'/SMC_dataset1/Spectrograms1024.csv'

x = np.array(pd.read_csv(SpectrogramPath, sep=',' ,header=None))
x = x[0:8,:]
print(x.shape)

(8, 1024)


In [105]:
ltsm_name = 'Results/120Result.npy'


for i in range(int(len(x))):
    #x = np.load(ltsm_name)
    element = np.random.randint(0,len(x))
    x = np.reshape(x, (-1, 1024))
    inputdata = x[i,:]
    inputdata = np.reshape(inputdata, (1,-1))
    decoded_data = decoder.predict(inputdata)
    np.save('Results/Spectrogram/{:03d}'.format(i), decoded_data)
    mel_ae, y_ae = su.spectrogram_to_audio(decoded_data,n_mels,n_frames, sr, n_fft, hop_length, fmin, fmax)
    write_wav('Results/wav/{:03d}.wav'.format(i),y_ae, sr, norm=True)
    

In [96]:
ltsm_name = 'Results/120Result.npy'

x = np.load(ltsm_name)
x = np.reshape(x, (-1, 1024))
print(x.shape, len(x))

(960, 1024) 960


#Getting mean squared error

In [ ]:

Predicted = model.predict(train_x[IndexStart:IndexEnd, :, :])

y_sample = np.apply_along_axis(mdn.sample_from_output, 1, Predicted, 1024, N_MIXES, temp=1.0)
y_sample = np.squeeze(y_sample)
TrueValues = np.array(validate_y.iloc[IndexStart:IndexEnd, :])

Predicted = y_sample.reshape(8192, 1)
TrueValues = TrueValues.reshape(8192, 1)
np.save('Results/Predicted_Vector_Unquantized', Predicted)
np.save('Results/TrueValues', TrueValues)


#Standard absolute error
Error = mean_absolute_error(TrueValues, Predicted)

#Error with all values rounded to nearest in
PredictedQuantized = np.rint(Predicted)
QuantizedError = mean_absolute_error(TrueValues, PredictedQuantized)


#Only values less than 1 are quantized 

for n in range(np.size(Predicted, 0)):
    for m in range (np.size(Predicted, 1)):
        if Predicted[n][m] < 1:
            Predicted[n][m] = 0
           

#Predicted = Predicted.reshape(8,256)
#np.save('Predicted_Vector_PartialQuantized', Predicted)
PartialQuantizedError = mean_absolute_error(TrueValues, Predicted)


print(Error, PartialQuantizedError, QuantizedError)
np.save('Results/Predicted_Vector_Quantized', PredictedQuantized)
np.save('Results/Predicted_Vector_PartialQuantized', Predicted)